# Train pHLA predictor with a K-fold strategy and make predictions on a test set

In [1]:
import pandas as pd
from main import StriMap_pHLA
from utils import load_train_data
from sklearn.model_selection import train_test_split

### Load data

We provided only a small subset here for demonstration purposes. For full dataset, please refer to the [Zenodo repository](https://zenodo.org/records/18002170)

In [2]:
df = pd.read_csv("examples/phla_train_set_example.csv")

### 5-fold train/val splits

In [3]:
num_fold = 5
train_folds, val_folds = zip(*[
    train_test_split(
        df, test_size=0.1, random_state=i, stratify=df["label"]
    )
    for i in range(num_fold)
])

train_folds = [d.reset_index(drop=True) for d in train_folds]
val_folds   = [d.reset_index(drop=True) for d in val_folds]

### HLA full sequence mapping

In [4]:
train_folds, val_folds = load_train_data(
    df_train_list=list(train_folds),
    df_val_list=list(val_folds),
    hla_dict_path="HLA_dict.npy",
)

Loading training and validation data...


### Initialize StriMap pHLA model

In [5]:
strimap = StriMap_pHLA(
    device="cuda:0",
    model_save_path="params/phla_model.pt",
    cache_dir="cache",
)

# Prepare embeddings (cached for faster training)
strimap.prepare_embeddings(
    pd.concat([*train_folds, *val_folds], ignore_index=True), 
    force_recompute=False,
)

Initializing encoders...
✓ Loaded 20 AAindex features
Initializing binding prediction model...
✓ StriMap initialized on cuda:0

Preparing embeddings for:
  - 1659 unique peptides
  - 113 unique HLAs



Phys encoding: 100%|██████████| 1/1 [00:00<00:00, 213.17it/s]

[ESM2] Loading cached embeddings from /ewsc/cao/StriMap/strimap-tools/cache/pep_esm2_layer33.pt



INFO:model:Loading /ewsc/cao/StriMap/strimap-tools/cache/hla_feat_dict.pt and /ewsc/cao/StriMap/strimap-tools/cache/hla_coord_dict.pt


[ESM2] No new sequences for pep, using existing cache
[ESM2] Loading cached embeddings from /ewsc/cao/StriMap/strimap-tools/cache/hla_esm2_layer33.pt
[ESM2] No new sequences for hla, using existing cache


INFO:model:Saved /ewsc/cao/StriMap/strimap-tools/cache/hla_feat_dict.pt and /ewsc/cao/StriMap/strimap-tools/cache/hla_coord_dict.pt
INFO:model:No new hla sequences found


✓ All embeddings prepared!
  - Phys: 1659 peptides, 113 HLAs
  - ESM2: 1659 peptides, 113 HLAs
  - Struct: 113 HLAs



### K-fold training

In [6]:
all_history = strimap.train_kfold(
    train_folds=list(zip(train_folds, val_folds)),
    epochs=10, # Reduced for demonstration; use 100 for actual training
)


Starting 5-Fold Cross-Validation Training

Training Fold 1/5
Train: 1494 samples | Val: 166 samples
Creating datasets...

Starting training for 10 epochs [Fold 0]...


[Fold 0] Epoch [1/10] | Train Loss: 0.0747 | Val Loss: 0.0685 | Val AUC: 0.7839 | Val PRC: 0.4513
Validation improved → Saving model (Score=0.4513) to params/phla_model_fold0.pt


[Fold 0] Epoch [2/10] | Train Loss: 0.0671 | Val Loss: 0.0627 | Val AUC: 0.7922 | Val PRC: 0.5881
Validation improved → Saving model (Score=0.5881) to params/phla_model_fold0.pt


[Fold 0] Epoch [3/10] | Train Loss: 0.0593 | Val Loss: 0.0539 | Val AUC: 0.7653 | Val PRC: 0.4850
EarlyStopping counter: 1 out of 5


[Fold 0] Epoch [4/10] | Train Loss: 0.0552 | Val Loss: 0.0514 | Val AUC: 0.7653 | Val PRC: 0.6220
Validation improved → Saving model (Score=0.6220) to params/phla_model_fold0.pt


...

[Fold 4] Epoch [6/10] | Train Loss: 0.0504 | Val Loss: 0.0476 | Val AUC: 0.8100 | Val PRC: 0.6547
Validation improved → Saving model (Score=0.6547) to params/phla_model_fold4.pt


[Fold 4] Epoch [7/10] | Train Loss: 0.0480 | Val Loss: 0.0455 | Val AUC: 0.8235 | Val PRC: 0.6892
Validation improved → Saving model (Score=0.6892) to params/phla_model_fold4.pt


[Fold 4] Epoch [8/10] | Train Loss: 0.0437 | Val Loss: 0.0430 | Val AUC: 0.8372 | Val PRC: 0.6735
EarlyStopping counter: 1 out of 5


[Fold 4] Epoch [9/10] | Train Loss: 0.0433 | Val Loss: 0.0435 | Val AUC: 0.8341 | Val PRC: 0.6749
EarlyStopping counter: 2 out of 5


[Fold 4] Epoch [10/10] | Train Loss: 0.0423 | Val Loss: 0.0400 | Val AUC: 0.8551 | Val PRC: 0.7167
Validation improved → Saving model (Score=0.7167) to params/phla_model_fold4.pt

[Fold 4] Loading best model from params/phla_model_fold4.pt...
✓ Training completed for Fold 4!

✓ All 5 folds training completed!

Cross-Validation Summary:
----------------------------------------------------------------------
Fold 0: Best Val AUC = 0.8385 (Epoch 10)
Fold 1: Best Val AUC = 0.8678 (Epoch 10)
Fold 2: Best Val AUC = 0.7208 (Epoch 9)
Fold 3: Best Val AUC = 0.8098 (Epoch 8)
Fold 4: Best Val AUC = 0.8551 (Epoch 10)
----------------------------------------------------------------------
Mean Val AUC: 0.8184 ± 0.0525



# Predict

In [10]:
from main import StriMap_pHLA
from utils import load_test_data
import pandas as pd

# Load test data
df_test = pd.read_csv("examples/phla_test_set_example.csv")

if 'label' not in df_test.columns:
    df_test['label'] = 0  # Dummy label column for compatibility

# Standardize HLA fields and map alleles
df_test = load_test_data(
    df_test=df_test,
    hla_dict_path="HLA_dict.npy",
)

# Initialize StriMap with a trained checkpoint
strimap = StriMap_pHLA(
    device="cuda:0",  # or "cpu"
    model_save_path=f"params/phla_model.pt", # Path to trained model
    cache_dir="cache", # Cache directory for embeddings
)

# Prepare embeddings (cached for faster inference)
strimap.prepare_embeddings(
    df_test,
    force_recompute=False,
)

# Run prediction
# y_prob_test, _ = strimap.predict(df_test, use_kfold=True, num_folds=5)

# Run evaluation if ground truth labels are available
y_prob_test, _ = strimap.evaluate(df_test, use_kfold=True, num_folds=5)

df_test["predicted_score"] = y_prob_test
print(df_test.head())

Processing test data...
✓ Test set: 416 samples
Initializing encoders...
✓ Loaded 20 AAindex features
Initializing binding prediction model...
✓ StriMap initialized on cuda:0

Preparing embeddings for:
  - 416 unique peptides
  - 97 unique HLAs



Phys encoding: 100%|██████████| 1/1 [00:00<00:00, 266.17it/s]

[ESM2] Loading cached embeddings from /ewsc/cao/StriMap/strimap-tools/cache/pep_esm2_layer33.pt


[ESM2] No new sequences for pep, using existing cache
[ESM2] Loading cached embeddings from /ewsc/cao/StriMap/strimap-tools/cache/hla_esm2_layer33.pt


INFO:model:Loading /ewsc/cao/StriMap/strimap-tools/cache/hla_feat_dict.pt and /ewsc/cao/StriMap/strimap-tools/cache/hla_coord_dict.pt
INFO:model:Saved /ewsc/cao/StriMap/strimap-tools/cache/hla_feat_dict.pt and /ewsc/cao/StriMap/strimap-tools/cache/hla_coord_dict.pt
INFO:model:No new hla sequences found


...

Loading model from params/phla_model_fold3.pt...


Predicting: 100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


Loading model from params/phla_model_fold4.pt...


Predicting: 100%|██████████| 2/2 [00:01<00:00,  1.59it/s]

✓ Ensemble prediction completed using 5 models

Evaluation Results [5-Fold Ensemble (mean)]
tn = 331, fp = 16, fn = 41, tp = 28
y_pred: 0 = 372 | 1 = 44
y_true: 0 = 347 | 1 = 69
AUC: 0.8539 | PRC: 0.6072 | ACC: 0.8630 | MCC: 0.4350 | F1: 0.4956
Precision: 0.6364 | Recall: 0.4058

           HLA      peptide  label  \
0  HLA-B*27:08  GKPAETIRIGD      0   
1  HLA-B*40:01   TEYEEAQDAI      0   
2  HLA-A*68:02   NAENEFVTIK      1   
3  HLA-C*07:01  YKTDVEQIKIN      0   
4  HLA-C*07:01    NGICIYFSR      0   

                                            HLA_full  predicted_score  
0  SHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREEP...         0.264067  
1  SHSMRYFHTAMSRPGRGEPRFITVGYVDDTLFVRFDSDATSPRKEP...         0.394951  
2  SHSMRYFYTSMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEP...         0.461650  
3  SHSMRYFDTAVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPRGEP...         0.308959  
4  SHSMRYFDTAVSRPGRGEPRFISVGYVDDTQFVRFDSDAASPRGEP...         0.317483  
